## MobileNet V1 model for training Food101 datasets
* number of parameters: 3,380,965

In [1]:
import torch

from torchvision import transforms, datasets

from pathlib import Path

import sys
sys.path.append('..')

from python_scripts import model_builder, engine, data_setup

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
food101_transforms_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

food101_transforms_test = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

In [4]:
learning_rate_list = [1e-3, 1e-4]
weight_decay_list = [1e-4]
epochs_list = [5]
batch_size_list = [16]

In [5]:
data_dir = Path('../data/extracted')

train_data_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=food101_transforms_train,
    download=True
)
test_data_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=food101_transforms_test,
    download=True
)

class_names_food101 = train_data_food101.classes

  1%|          | 48103424/4996278331 [00:48<1:22:40, 997501.43it/s] 


KeyboardInterrupt: 

In [ ]:
def model_generator(weights=None):
    return model_builder.MobileNetV1(num_classes=len(class_names_food101))

In [ ]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=0.1, # you can start with small number of train set to check normality at first
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=0.1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=None,
    model_generator=model_generator,
    model_weights=None,
    model_name='MobileNetV1_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=False,
    device=device,
    gradient_accumulation_num=1
)